In [5]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping # type: ignore
import keras.backend as K # type: ignore
from keras.models import Sequential # type: ignore
from keras.layers import LSTM,Dense, Bidirectional, GRU, Dropout, BatchNormalization, SimpleRNN # type: ignore
from keras.regularizers import l2 # type: ignore
import matplotlib.pyplot as plt
import os
from tensorboard import notebook
from tqdm import tqdm
import pickle as pkl
import warnings
import longitudinal_tadpole.pipeline as p
from longitudinal_tadpole.threshmax import ThreshMax

In [ ]:
cwd = os.getcwd()
with open(f"/Users/susmaa01/Documents/eipy/longitudinal_tadpole/tadpole_data/tadpole_data_time_imptn_norm_thrshld30.pickle", "rb") as file:
    data_nested_dict = pkl.load(file)
with open(f"/Users/susmaa01/Documents/eipy/longitudinal_tadpole/tadpole_data/tadpole_labels_time_imptn_norm_thrshld30.pickle", "rb") as file:
    labels = pkl.load(file)

In [ ]:
bps = []
for i in range(5):
    with open(f"/Users/susmaa01/Documents/eipy/longitudinal_tadpole/base_predictions/multiclass/data_at_n_w_labels_at_n/no_sampling/split_{i}.pkl", "rb") as file:
        split = pkl.load(file)
        bps.append(split)

# TIME SERIES TIME

In [ ]:
def ohe(y):
    labels_across_time = np.eye(3)[y]

    return labels_across_time

In [ ]:
def get_class_weights_for_t(training_labels):
    class_weights_for_labels = []
    for t in range(training_labels.shape[1]):
        bin_counts_per_t = 1 / np.bincount(training_labels[:,t])
        norm_bin_counts_per_t = bin_counts_per_t/np.sum(bin_counts_per_t)
        class_weights_t = dict(zip(range(3), norm_bin_counts_per_t))
        class_weights_for_labels.append(class_weights_t)
    return class_weights_for_labels
    # mapped_array = [np.array([class_weights_for_labels[t][entry] for entry in training_labels[:,t]]) for t in range(training_labels.shape[1])]
    # mapped_array = np.stack(mapped_array, axis=1)
    # return tf.constant(mapped_array, dtype='float32')

In [ ]:
#ordinal categorical crossentropy. Weighs error by how many classes output was off by. weights range from 1 to 2. assigns class weights across time.
# https://stats.stackexchange.com/questions/87826/machine-learning-with-ordered-labels
#gamma = d(MCI,CN) - d(MCI,Dementia) = d(MCI,CN) - 1

def occ_loss(gamma=0):
    global class_weights
    def loss(y_true, y_pred):
        y_true_ord = tf.argmax(y_true, axis=-1)
        y_pred_ord = tf.argmax(y_pred, axis=-1)

        losses_over_time = []
        for t in range(y_true.shape[1]):
            y_true_ord_t = y_true_ord[:,t]
            
            class_weights_t = class_weights[t]
            w_c_t = tf.gather(tf.constant(list(class_weights_t.values()), dtype=tf.float32),
                                y_true_ord_t)

            if gamma=='none':
                loss = tf.keras.losses.categorical_crossentropy(y_true[:,t], y_pred[:,t]) * w_c_t
            else:
                y_true_ord_gamma = y_true_ord_t + tf.cast(y_true_ord_t != 0, tf.int64) * gamma
                y_pred_ord_gamma = y_pred_ord[:,t] + tf.cast(y_pred_ord[:,t] != 0, tf.int64) * gamma
                w_o_t = tf.cast(tf.abs(y_true_ord_gamma - y_pred_ord_gamma) / (2 + gamma), dtype='float32') + 1
    
                loss = tf.keras.losses.categorical_crossentropy(y_true[:,t], y_pred[:,t]) * w_o_t * w_c_t

            losses_over_time.append(loss)
        return tf.reduce_mean(tf.stack(losses_over_time, axis=-1), axis=-1)
    return loss

In [ ]:
def dem_prev(label_seq):
    return [np.sum(y==2) for y in label_seq]

In [ ]:
def reformat_data(dfs):
    #reformat labels
    labels_across_time = np.column_stack([df['labels'].values for df in dfs])
    labels_across_time = np.eye(3)[labels_across_time]
    
    # reformat data
    RNN_training_data_fold = [df.drop(columns=["labels"], axis=1, level=0) for df in dfs]
    data_arrays_per_timepoint = [df.to_numpy() for df in RNN_training_data_fold]
    tensor_3d = np.stack(data_arrays_per_timepoint, axis=1)

    return tensor_3d, labels_across_time

In [ ]:
def fit_restandardize(data):
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    
    #for restandardizing bl class 2 columns to be all -1
    means[means==0] = 1
    stds[stds == 0] = 1

    restandardized_data = (data-means)/stds

    return restandardized_data, means, stds

def restandardize(data, means, stds):

    return (data-means)/stds

train lstms

In [ ]:
#use data up to time point n, 
#e.g., when n=3 means use data from t=0,1,2 and predict on label from t=3
cell_list = ["LSTM"]##, "GRU"]#, 'RNN', "biLSTM", "biGRU"]
def train_eval_RNNs(train, test, seed, cells=cell_list, decision='argmax', random_bl=False, sampling=None, gamma=0):
    global f_scores_test
    global p_scores_test
    global r_scores_test
    # global f_scores_train
    for cell in cells:
        y_preds_test = []
        y_tests = []
        
        y_preds_train = []
        y_trains = []

        y_preds_val = []
        y_vals = []
        
        for fold, dfs in enumerate(train):
            if random_bl==False:
                real="Real"
            else:
                real="Random"
            print(real, cell, "seed",seed+1, "fold", fold+1, "gamma=",gamma)
            
            X_train, y_train = reformat_data(dfs)
            X_test, y_test = reformat_data(test[fold])


            X_train, X_test = X_train[:, :-1, :], X_test[:, :-1, :] #data excluding last time point
            y_train, y_test = y_train[:, 1:, :], y_test[:, 1:, :] #labels excluding baseline


            #restandardize to have mean=0, std=1 (for tanh)
            X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.09,
                                                                stratify=dem_prev(y_train), random_state=seed**2)
            X_train, means, stds = fit_restandardize(X_train)
            X_test, X_val = restandardize(X_test, means=means, stds=stds), restandardize(X_val, means=means, stds=stds)
            

            
            y_trains.append(y_train)
            y_tests.append(y_test)
            y_vals.append(y_val)

            
            y_train_ord = np.argmax(y_train, axis=-1)
            global class_weights
            class_weights = [dict(zip(range(3), compute_class_weight(class_weight='balanced', classes=[0,1,2], y=y_train_ord[:,t]))) for t in range(y_train_ord.shape[1])]
            
            model = p.build_RNN(units=64, cell=cell, drout=0.2, deep=True, L2=0.00,
                               activation='tanh', reg_layer='batchnorm', gamma=gamma)
            
            early_stop = EarlyStopping(
                monitor='val_loss', patience=30, verbose=1,
                restore_best_weights=True, start_from_epoch=10)
            
            
            if random_bl:
                np.random.shuffle(y_train)
                np.random.shuffle(y_val)

            #fit model
            model.fit(X_train, y_train, epochs=2000, validation_data=[X_val, y_val],
                       verbose=1, callbacks=[early_stop])

            y_preds_train.append(model.predict(X_train))
            y_preds_test.append(model.predict(X_test))
            y_preds_val.append(model.predict(X_val))
        
        y_pred_train = np.concatenate(y_preds_train, axis=0)
        y_train = np.concatenate(y_trains, axis=0)

        y_pred_test = np.concatenate(y_preds_test, axis=0)
        y_test = np.concatenate(y_tests, axis=0)
       
        y_pred_val = np.concatenate(y_preds_val, axis=0)
        y_val = np.concatenate(y_vals, axis=0)

        if decision == "argmax": # for doing traditional decision making
            y_test_ord = np.argmax(y_test, axis=-1)
            f_scores_cell_seed_across_time_test = np.stack([f1_score(y_test_ord[:,i], np.array([np.argmax(y_hat) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])
            p_scores_cell_seed_across_time_test = np.stack([precision_score(y_test_ord[:,i], np.array([np.argmax(y_hat) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])
            r_scores_cell_seed_across_time_test = np.stack([recall_score(y_test_ord[:,i], np.array([np.argmax(y_hat) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])


        elif decision == 'threshmax':
            y_train_ord = np.argmax(y_train, axis=-1)
            y_test_ord = np.argmax(y_test, axis=-1)
            y_val_ord = np.argmax(y_val, axis=-1)
            
            y_not_test_ord = np.concatenate([y_train_ord, y_val_ord], axis=0)
            y_pred_not_test = np.concatenate([y_pred_train, y_pred_val], axis=0)

            tm = ThreshMax(classes=[0,1,2], thresh_class=1, class_to_optimize='avg')

            thresholds = [tm.find_tmax(y_trues=y_not_test_ord[:,i], y_preds=y_pred_not_test[:,i,:]) for i in range(y_not_test_ord.shape[1])]
            f_scores_cell_seed_across_time_test = np.stack([f1_score(y_test_ord[:,i], np.array([tm.compute_threshmax(y_hat, thresholds[i]) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])
            p_scores_cell_seed_across_time_test = np.stack([precision_score(y_test_ord[:,i], np.array([tm.compute_threshmax(y_hat, thresholds[i]) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])
            r_scores_cell_seed_across_time_test = np.stack([recall_score(y_test_ord[:,i], np.array([tm.compute_threshmax(y_hat, thresholds[i]) for y_hat in y_pred_test[:,i,:]]), average=None) for i in range(y_test.shape[1])])
        
        f_scores_test[f'{real} {cell}'].append(f_scores_cell_seed_across_time_test)
        p_scores_test[f'{real} {cell}'].append(p_scores_cell_seed_across_time_test)
        r_scores_test[f'{real} {cell}'].append(r_scores_cell_seed_across_time_test)

In [ ]:
f_scores_test = dict()
p_scores_test = dict()
r_scores_test = dict()
y_test_predictions = dict()
for cell in cell_list:
    for real in ['Real']:#, 'Random']:
        f_scores_test[f'{real} {cell}'] = []
        p_scores_test[f'{real} {cell}'] = []
        r_scores_test[f'{real} {cell}'] = []
        y_test_predictions[f'{real} {cell}'] = []

# f_scores_train = {k: [] for k in cell_list} #gets score distribution for diff cells across splits
for seed in range(5): #range(len(bps)):
    for gamma in range(5):
        train_eval_RNNs(train=bps[seed][0], test=bps[seed][1], seed=seed, decision='threshmax', random_bl=False, sampling=None, gamma=gamma)
    # train_eval_RNNs(train=bps[seed][0], test=bps[seed][1], seed=seed, decision='argmax', random_bl=True, sampling='dem_prev')

f_score_arrays_test = {k: np.stack(v, axis=0) for k,v in f_scores_test.items()}
p_score_arrays_test = {k: np.stack(v, axis=0) for k,v in p_scores_test.items()}
r_score_arrays_test = {k: np.stack(v, axis=0) for k,v in r_scores_test.items()}

In [ ]:
f_scores_gamma = { f'gamma={gamma}': np.stack([item for index, item in enumerate(f_scores_test['Real LSTM']) if index % 5 == gamma], axis=0) for gamma in range(5)}
p_scores_gamma = { f'gamma={gamma}': np.stack([item for index, item in enumerate(p_scores_test['Real LSTM']) if index % 5 == gamma], axis=0) for gamma in range(5)}
r_scores_gamma = { f'gamma={gamma}': np.stack([item for index, item in enumerate(r_scores_test['Real LSTM']) if index % 5 == gamma], axis=0) for gamma in range(5)}

for seed in range(5): #range(len(bps)):
    train_eval_RNNs(train=bps[seed][0], test=bps[seed][1], seed=seed, decision='threshmax', random_bl=False, sampling=None, gamma='none')

In [ ]:
f_scores_gamma['no gamma'] = np.stack(f_scores_test['Real LSTM'][-5:], axis=0)
p_scores_gamma['no gamma'] = np.stack(p_scores_test['Real LSTM'][-5:], axis=0)
r_scores_gamma['no gamma'] = np.stack(r_scores_test['Real LSTM'][-5:], axis=0)
gamma_results = {'f': f_scores_gamma, 'p': p_scores_gamma, 'r': r_scores_gamma}    